<a href="https://colab.research.google.com/github/MuhammadIrzam447/NewEncodings/blob/main/Train_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/MMLearning/data/food-101/flip/multimodal_img_et_flip

/content/MMLearning/data/food-101/flip/multimodal_img_et_flip


In [ ]:
# Food-101 New Encoding ResNet-101

In [2]:
from google.colab import drive
import pandas as pd
import os
from PIL import Image
import torch
import torchvision
from torchvision import datasets, transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import UnidentifiedImageError
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


# Loading Training Dataset and Preprocessing

In [4]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.dataset = datasets.ImageFolder(data_dir)
        self.classes = sorted(os.listdir(data_dir))
        self.class_lengths = self._compute_class_lengths()
        self.num_classes = len(self.dataset.classes)

    def _compute_class_lengths(self):
        class_lengths = {cls: 0 for cls in self.classes}

        for cls in self.classes:
            cls_dir = os.path.join(self.data_dir, cls)
            if os.path.isdir(cls_dir):
                class_lengths[cls] = len(os.listdir(cls_dir))

        return class_lengths

    def __getitem__(self, index):
        image, label = self.dataset[index]

        # Apply additional transform for the test dataset if provided
        if self.transform:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.dataset)

    def get_num_classes(self):
        return self.num_classes


In [6]:
data_dir = "./train"
dataset = CustomDataset(data_dir, transform=train_transform)

In [7]:
print("Number of samples:", len(dataset))
print("Number of classes:", len(dataset.classes))

Number of samples: 203964
Number of classes: 101


In [8]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [9]:
data_dir1 = "./test"
dataset1 = CustomDataset(data_dir1, transform=test_transform)

In [10]:
print("Number of samples:", len(dataset1))
print("Number of classes:", len(dataset1.classes))

Number of samples: 45432
Number of classes: 101


In [ ]:
# from torchvision.datasets import ImageFolder
# from torch.utils.data import ConcatDataset
# combined_dataset = ConcatDataset([dataset1, dataset])

In [ ]:
# print("Number of samples:", len(combined_dataset))
# # print("Number of classes:", len(combined_dataset.classes))

In [11]:
batch_size = 32
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
validation_data_loader = torch.utils.data.DataLoader(dataset1, batch_size=batch_size, shuffle=False)

In [12]:
len(data_loader), len(validation_data_loader)

(6374, 1420)

# EDA

In [ ]:
class_lengths_dict = dataset.class_lengths
total_sum = sum(class_lengths_dict.values())
dict_length = len(class_lengths_dict)

# Print the length
print("Dictionary length:", dict_length)
# Print the total sum
print("Total sum:", total_sum)
print(class_lengths_dict.values())

In [ ]:
# Extract class labels and counts from the dictionary
class_labels = list(class_lengths_dict.keys())
class_counts = list(class_lengths_dict.values())

# Create a count plot
plt.figure(figsize=(4, 5))
sns.barplot(x=class_labels, y=class_counts)
plt.xticks(rotation=90)
plt.xlabel('Class')
plt.ylabel('Number of samples')
plt.title('Count of Instances in Each Class')
plt.show()

# Loading ResNet-50

In [13]:
num_classes = dataset.get_num_classes()
print("Number of classes:", num_classes)

Number of classes: 101


In [ ]:
resnet = torchvision.models.resnet101(pretrained=True)
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, num_classes)  # num_classes is the number of classes in your dataset
resnet.to(device)
print(resnet)

# Fine Tuning

In [15]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
num_epochs = 30

In [16]:
from keras.metrics import top_k_categorical_accuracy
from keras import backend as K
from sklearn.metrics import roc_auc_score
training_loss = []

for epoch in range(num_epochs):
    # Set the model to training mode
    resnet.train()
    running_loss = 0.0

    # Iterate over the data loader
    for images, labels in data_loader:
        # Move the images and labels to the GPU if available
        images = images.to(device)
        labels = labels.to(device)

        # Clear the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = resnet(images)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Update the running loss
        running_loss += loss.item() * images.size(0)

    # Calculate the average loss for the epoch
    epoch_loss = running_loss / len(data_loader.dataset)
    training_loss.append(epoch_loss)
    # Print the epoch loss
    # print(f"Epoch {epoch+1}/{num_epochs} Training Loss: {epoch_loss:.4f}")

    resnet.eval()

    predicted_classes = []
    actual_labels = []

    # all_outputs = []


    with torch.no_grad():
        for images, labels in validation_data_loader:
            # Move the images and labels to the GPU if available
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = resnet(images)

            # Get the predicted labels
            _, predicted_label = torch.max(outputs, 1)

            # Store the predicted and true labels
            predicted_classes.extend(predicted_label.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())

            # all_outputs.append(outputs.cpu().numpy())

    accuracy = accuracy_score(actual_labels, predicted_classes)
    # precision = precision_score(actual_labels, predicted_classes, average='weighted')
    # recall = recall_score(actual_labels, predicted_classes, average='weighted')
    # f1 = f1_score(actual_labels, predicted_classes, average='weighted')

    # top_5_accuracy = top_k_categorical_accuracy(actual_labels.astype('float32'), predicted_classes.astype('float32'), k=5)
    # top_5_accuracy_value = K.eval(K.mean(top_5_accuracy))

    # auroc = roc_auc_score(actual_labels, predicted_classes)

    # actual_labels_tensor = torch.tensor(actual_labels)
    # predicted_classes_tensor = torch.tensor(predicted_classes)
    # all_outputs = np.concatenate(all_outputs, axis=0)
    # all_outputs_tensor = torch.tensor(all_outputs)
    # _, top5_predicted_indices = torch.topk(all_outputs_tensor, k=5, dim=1)
    # correct_top5 = torch.any(top5_predicted_indices == actual_labels_tensor.view(-1, 1), dim=1)
    # top5_accuracy = torch.mean(correct_top5.float()).item()

    # print("Accuracy:", accuracy)
    # print("Precision:", precision)
    # print("Recall:", recall)
    # print("F1-score:", f1)
    # print(f"Epoch: {epoch+1}/{num_epochs} ========> Training Loss: {epoch_loss:.4f}  Accuracy: {accuracy:.4f} Top-5 Accuracy: {top5_accuracy:.4f}")

    print(f"Epoch: {epoch+1}/{num_epochs} ========> Training Loss: {epoch_loss:.4f}  Accuracy: {accuracy:.4f}")
    save_dir = "/content/MMLearning/data/Models/Model-21"

    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist
    model_name = str(epoch+1) + "_model.pth"
    save_path = os.path.join(save_dir, model_name)  # Specify the complete path to the model file
    torch.save(resnet.state_dict(), save_path)

Epoch: 1/30 ========> Training Loss: 3.3141  Accuracy: 0.4957
Epoch: 2/30 ========> Training Loss: 2.2525  Accuracy: 0.5668
Epoch: 3/30 ========> Training Loss: 1.8652  Accuracy: 0.6079
Epoch: 4/30 ========> Training Loss: 1.5693  Accuracy: 0.6286
Epoch: 5/30 ========> Training Loss: 1.2919  Accuracy: 0.6390
Epoch: 6/30 ========> Training Loss: 1.0325  Accuracy: 0.6458
Epoch: 7/30 ========> Training Loss: 0.7991  Accuracy: 0.6399
Epoch: 8/30 ========> Training Loss: 0.6171  Accuracy: 0.6450
Epoch: 9/30 ========> Training Loss: 0.4918  Accuracy: 0.6357
Epoch: 10/30 ========> Training Loss: 0.4087  Accuracy: 0.6331
Epoch: 11/30 ========> Training Loss: 0.3562  Accuracy: 0.6477
Epoch: 12/30 ========> Training Loss: 0.3213  Accuracy: 0.6409
Epoch: 13/30 ========> Training Loss: 0.2911  Accuracy: 0.6406
Epoch: 14/30 ========> Training Loss: 0.2729  Accuracy: 0.6432
Epoch: 15/30 ========> Training Loss: 0.2553  Accuracy: 0.6465
Epoch: 16/30 ========> Training Loss: 0.2426  Accuracy: 0.6441
E

KeyboardInterrupt: 

In [ ]:
# Plot the loss curve
plt.plot(range(1, num_epochs+1), training_loss)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.grid(True)
plt.show()

# Save Best Model File to Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cd /content/Models

In [ ]:
# !cp 7_model.pth /content/drive/MyDrive/filename

In [ ]:
save_dir = './Models/Models_fused_food101/'
load_path = os.path.join(save_dir, "22_model (2).pth")

# Create an instance of the ResNet model
resnet = torchvision.models.resnet101(pretrained=False)
resnet.fc = nn.Linear(2048, 101) # Choose the number of output classses as per your model

# Load the saved model parameters
resnet.load_state_dict(torch.load(load_path))
# resnet.load_state_dict(torch.load(load_path, map_location=torch.device('cpu')))

# Set the model to evaluation mode and respective device
resnet.eval()
resnet.to(device)

In [ ]:
resnet.eval()

predicted_classes = []
actual_labels = []

with torch.no_grad():
    for images, labels in validation_data_loader:

        images = images.to(device)
        labels = labels.to(device)


        outputs = resnet(images)
        _, predicted_label = torch.max(outputs, 1)

        predicted_classes.extend(predicted_label.cpu().tolist())
        actual_labels.extend(labels.cpu().tolist())

accuracy = accuracy_score(actual_labels, predicted_classes)

In [ ]:
print(f"Accuracy: {accuracy:.4f}")